# Read data

In [8]:
import pandas as pd

data_dir = "data/对话文本.csv"

df = pd.read_csv(data_dir)
df.head()

,param_id,contact_id,content,extracted_content
0,288,3ee4b9f4-7674-4978-9066-d89b46c9adb4,"agent: 主持人,多謝您的訪問,我們下集見,再見!\ncustomer: 喂,嗨,嗨,我...","_______costTime:21508ms;{""statusCode"": 200, ""b..."
1,289,9bdd9ccf-d94b-4f38-8a5d-35e4dd43aeee,"agent: 咁嘅\ncustomer: 你好呀,我正在用你們的抽濕機\ncustomer:...","_______costTime:13034ms;{""statusCode"": 200, ""b..."
2,290,8fcf8eb4-d4da-4146-a42c-d3ba711b4ed0,customer: 夭壽\nagent: 哦\ncustomer: 雞腿\ncustomer...,"_______costTime:16609ms;{""statusCode"": 200, ""b..."
3,291,aec95ea8-e448-45a6-be21-63a6b6326b9f,agent: 嗨\ncustomer: 你好呀\ncustomer: 今日呢就買咗架洗衣機嘅...,"_______costTime:12588ms;{""statusCode"": 200, ""b..."
4,292,942df28b-6803-4795-ab8b-7cba16a92ea8,"customer: 喂,駱小姐你好。\nagent: 早晨鐘之後會唔會報我姓駱,請問有咩幫到...","_______costTime:19305ms;{""errorMessage"": ""subs..."


# Extract necessary info

In [9]:
prompt = f"""下面是一段agent与customer的对话
{df.loc[1, 'content']}

请从上面的对话中抽取如下信息，并以json格式返回，如果对话中没有提到相应字段的内容，则填""：
 {{"customerType": "个人", "customerName": "溫先生", "phoneNumber1": "", "phoneNumber2": "", "email": "", "address": "楊逸居第三座,九樓,A7", "productBrand": "Toshiba", "productCategoryName": "", "serialNumber": "ER-GD400HK", "srType": "维修", "srSubType": "维修", "symptomDescription": "燈膽燒了", "customerRequest": "維修法", "refNo": "", "selloutInvoiceNum": "", "salesDealerName": "", "installDealerName": ""}}
 
注意只返回抽取的json格式的结果，不返回其它额外信息。 
"""
prompt

'下面是一段agent与customer的对话\nagent: 咁嘅\ncustomer: 你好呀,我正在用你們的抽濕機\ncustomer: 用咗半年到啦咁而家呢佢壞咗呀佢呢\nagent: 嗨\ncustomer: 濕濕咗嘅時候呢嗰啲水呢直接流落去地下\ncustomer: 接不到那些水\nagent: 請問點稱呼呀?\nagent: 余先生,請問你可否給我抽濕口罩?\ncustomer: 我姓余的\nagent: 手機的型號\ncustomer: RADY200H\nagent: 多謝大家\ncustomer: Thank you for watching\nagent: OK\ncustomer: 拜拜\nagent: 又係咪check我嗰個水準\nagent: 放入去嘅時候,位置係正常\nagent: 唔好擺尾之類\ncustomer: 冇嘅冇排名\nagent: Check過冇問題嘅,如果係可以搵錢\nagent: 咁請問你嗰個抽濕機係買咗一年到?嘛係咪?\nagent: 上年九月抽濕機\ncustomer: 上年九月買的\nagent: 请问您的购买单和保用证是否存在?\ncustomer: 喺度嘅\nagent: 不存在的,如果我可以找师傅上来帮你看,麻烦你出示给他\nagent: 請問您的地址在哪裡?\ncustomer: 柴灣道111號\ncustomer: 高威港\nagent: 高威國幾座幾樓幾室\ncustomer: 第四座,四座八樓私宅\nagent: SipoCat\ncustomer: 係冇差\nagent: 麻煩你等一等我睇下如果係柴灣嘅可以幾時安排到過嚟先\ncustomer: 唔\nagent: 嚟緊星期二 23 號 12 點,\nagent: 屋企會唔會有忍\ncustomer: 哎呀\nagent: 咁我呢度幫你安排返啦係余生嘅到時上嚟之前師傅可以打返90997\nagent: 910 呢個電話高威國四座扮cat嘅啱啱\ncustomer: 我留兩個電話給你\nagent: 好呀你再講呀\ncustomer: 我太太梁小姐\nagent: 90256922,呢個係搵邊位??\nagent: 即係阿梁小姐\nagent: 打誰的電話先?\nagent: 由我講吧!\ncustomer: 你打這個先啦,我再留多個屋企電話比你啦\

## Qwen2-72B-Instruct-AWQ

In [30]:
#%pip install openai

In [31]:
# Set OpenAI's API key and API base to use vLLM's API server.
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
model_id = "Qwen2-72B-Instruct-AWQ"

stop_token_ids = ''
temp = 0.8

# Create an OpenAI client to interact with the API server
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def predict(message, history):
    # Convert chat history to OpenAI format
    history_openai_format = [{
        "role": "system",
        "content": "You are a great ai assistant."
    }]
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human})
        history_openai_format.append({
            "role": "assistant",
            "content": assistant
        })
    history_openai_format.append({"role": "user", "content": message})

    # Create a chat completion request and send it to the API server
    stream = client.chat.completions.create(
        model=model_id,  # Model name to use
        messages=history_openai_format,  # Chat history
        temperature=temp,  # Temperature for text generation
        # stream=True,  # Stream response
        extra_body={
            'repetition_penalty':
            1,
            'stop_token_ids': [
                int(id.strip()) for id in stop_token_ids.split(',')
                if id.strip()
            ] if stop_token_ids else []
        })

    return stream

In [7]:
# SageMaker Endpoint

from transformers import AutoTokenizer
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

endpoint_name = 'lmi-model-2024-06-11-09-18-56-099'
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer()
)

model_dir = '/home/ec2-user/SageMaker/efs/Models/Qwen2-72B-Instruct-AWQ'
tokenizer = AutoTokenizer.from_pretrained(model_dir)

def invoke_endpoint(prompt):

    # prompt = """'下面是一段agent与customer的对话\nagent: 咁嘅\ncustomer: 你好呀,我正在用你們的抽濕機\ncustomer: 用咗半年到啦咁而家呢佢壞咗呀佢呢\nagent: 嗨\ncustomer: 濕濕咗嘅時候呢嗰啲水呢直接流落去地下\ncustomer: 接不到那些水\nagent: 請問點稱呼呀?\nagent: 余先生,請問你可否給我抽濕口罩?\ncustomer: 我姓余的\nagent: 手機的型號\ncustomer: RADY200H\nagent: 多謝大家\ncustomer: Thank you for watching\nagent: OK\ncustomer: 拜拜\nagent: 又係咪check我嗰個水準\nagent: 放入去嘅時候,位置係正常\nagent: 唔好擺尾之類\ncustomer: 冇嘅冇排名\nagent: Check過冇問題嘅,如果係可以搵錢\nagent: 咁請問你嗰個抽濕機係買咗一年到?嘛係咪?\nagent: 上年九月抽濕機\ncustomer: 上年九月買的\nagent: 请问您的购买单和保用证是否存在?\ncustomer: 喺度嘅\nagent: 不存在的,如果我可以找师傅上来帮你看,麻烦你出示给他\nagent: 請問您的地址在哪裡?\ncustomer: 柴灣道111號\ncustomer: 高威港\nagent: 高威國幾座幾樓幾室\ncustomer: 第四座,四座八樓私宅\nagent: SipoCat\ncustomer: 係冇差\nagent: 麻煩你等一等我睇下如果係柴灣嘅可以幾時安排到過嚟先\ncustomer: 唔\nagent: 嚟緊星期二 23 號 12 點,\nagent: 屋企會唔會有忍\ncustomer: 哎呀\nagent: 咁我呢度幫你安排返啦係余生嘅到時上嚟之前師傅可以打返90997\nagent: 910 呢個電話高威國四座扮cat嘅啱啱\ncustomer: 我留兩個電話給你\nagent: 好呀你再講呀\ncustomer: 我太太梁小姐\nagent: 90256922,呢個係搵邊位??\nagent: 即係阿梁小姐\nagent: 打誰的電話先?\nagent: 由我講吧!\ncustomer: 你打這個先啦,我再留多個屋企電話比你啦\ncustomer: 以5687021\nagent: 屋企電話25687021,如果係到時打電話嘅時候,可以搵返梁小姐係\nagent: 902-56922\ncustomer: 係冇錯\nagent: 好得,我呢度幫你安排返\nagent: 23 號 12 點至 5 點過嚟\ncustomer: 星期五至五點,星期二。\nagent: 係,冇錯,有咩其他可以幫到你?\ncustomer: OK,好呀,\nagent: 都係打電話嚟,多謝!\ncustomer: 係冇?啦,\nagent: 拜拜\ncustomer: OK\n\n\n请从上面的对话中抽取如下信息，并以json格式返回，如果对话中没有提到相应字段的内容，则填""：\n {"customerType": "个人", "customerName": "溫先生", "phoneNumber1": "", "phoneNumber2": "", "email": "", "address": "楊逸居第三座,九樓,A7", "productBrand": "Toshiba", "productCategoryName": "", "serialNumber": "ER-GD400HK", "srType": "维修", "srSubType": "维修", "symptomDescription": "燈膽燒了", "customerRequest": "維修法", "refNo": "", "selloutInvoiceNum": "", "salesDealerName": "", "installDealerName": ""}\n \n注意只返回抽取的json格式的结果，不返回其它额外信息。 \n'"""
    system_prompt = "You are a helpful assistant."

    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # print(prompt)

    response = predictor.predict(
        {
            "inputs": prompt, 
             "parameters": 
             {
                 "max_new_tokens": 4096,
                 # Add any other sampling parameters as needed
                 "temperature": 0.7,
                 "top_k": 5,
                 "top_p": 0.9,
                 # "stop_token_ids": [], 
                 # "stop": ["\nASSISTANT", "\nUSER:"],
                 "include_stop_str_in_output": False,
                 # "skip_special_tokens": True,
                 "ignore_eos": False,
                 "repetition_penalty": 1,
             }
        }
    )
    
    generated_text = json.loads(response.decode('utf-8'))['generated_text']
    
    return generated_text

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from tqdm import tqdm
import time
import json

model_name = 'qwen2'

num_test = 100
for i, row in tqdm(df[:num_test].iterrows(), total=df[:num_test].shape[0]):
    content = row['content']
    
    prompt = f"""下面是一段agent与customer的对话
{content}

请从上面的对话中抽取如下信息，并以json格式返回，如果对话中没有提到相应字段的内容，则填""：
 {{"customerType": "个人", "customerName": "溫先生", "phoneNumber1": "", "phoneNumber2": "", "email": "", "address": "楊逸居第三座,九樓,A7", "productBrand": "Toshiba", "productCategoryName": "", "serialNumber": "ER-GD400HK", "srType": "维修", "srSubType": "维修", "symptomDescription": "燈膽燒了", "customerRequest": "維修法", "refNo": "", "selloutInvoiceNum": "", "salesDealerName": "", "installDealerName": ""}}
 
注意只返回抽取的json格式的结果，不返回其它额外信息。 
"""
    start_time = time.time()
    
    ## vllm direct deployment
    # response = predict(prompt, [])
    # generated_text = response.choices[0].message.content
    
    # sagemaker endpoint deployment
    generated_text = invoke_endpoint(prompt)
    
    time_cost = time.time() - start_time
    
    df.loc[i, f'extracted_info_{model_name}'] = generated_text
    res = json.loads(generated_text)
    for k, v in res.items():
        df.loc[i, f"{k}_{model_name}"] = v
    
    df.loc[i, f'time_cost_{model_name}'] = time_cost

 55%|█████▌    | 11/20 [01:26<01:03,  7.06s/it]

In [ ]:
df.head()

## Claude3-Sonnet

In [34]:
import boto3
import json
from botocore.exceptions import ClientError

model_name = 'sonnet'

bedrock = boto3.client(service_name='bedrock-runtime')

## only text
def request_claude(prompt, system_prompt, version='3-sonnet-20240229-v1:0'):
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192, 
        # "temperature": 0.1,
        # "top_p": 0.9,
        "system": system_prompt,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
    })

    modelId = f'anthropic.claude-{version}'
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    # print(f"response_body: {response_body}")
    
    return response_body.get('content')[0]['text']

In [35]:
from tqdm import tqdm
import time
import json

system_prompt = "You are a great ai assistant."

num_test = 100
for i, row in tqdm(df[:num_test].iterrows(), total=df[:num_test].shape[0]):
    content = row['content']
    
    prompt = f"""下面是一段agent与customer的对话
{content}

请从上面的对话中抽取如下信息，并以json格式返回，如果对话中没有提到相应字段的内容，则填""：
 {{"customerType": "个人", "customerName": "溫先生", "phoneNumber1": "", "phoneNumber2": "", "email": "", "address": "楊逸居第三座,九樓,A7", "productBrand": "Toshiba", "productCategoryName": "", "serialNumber": "ER-GD400HK", "srType": "维修", "srSubType": "维修", "symptomDescription": "燈膽燒了", "customerRequest": "維修法", "refNo": "", "selloutInvoiceNum": "", "salesDealerName": "", "installDealerName": ""}}
 
注意只返回抽取的json格式的结果，不返回其它额外信息。 
"""
    start_time = time.time()
    response = request_claude(prompt, system_prompt)
    
    time_cost = time.time() - start_time
    
#     print(f"############ prompt\n\n{prompt}")
#     print(f"############ response\n\n{response}")
#     print(f"############ response content\n\n{response.choices[0].message.content}")
    
    df.loc[i, f'extracted_info_{model_name}'] = response
    res = json.loads(response)
    for k, v in res.items():
        df.loc[i, f"{k}_{model_name}"] = v
    
    df.loc[i, f'time_cost_{model_name}'] = time_cost

100%|██████████| 20/20 [01:34<00:00,  4.71s/it]


In [36]:
df.head()

,param_id,contact_id,content,extracted_content,extracted_info_qwen2,customerType_qwen2,customerName_qwen2,phoneNumber1_qwen2,phoneNumber2_qwen2,email_qwen2,...,serialNumber_sonnet,srType_sonnet,srSubType_sonnet,symptomDescription_sonnet,customerRequest_sonnet,refNo_sonnet,selloutInvoiceNum_sonnet,salesDealerName_sonnet,installDealerName_sonnet,time_cost_sonnet
0,288,3ee4b9f4-7674-4978-9066-d89b46c9adb4,"agent: 主持人,多謝您的訪問,我們下集見,再見!\ncustomer: 喂,嗨,嗨,我...","_______costTime:21508ms;{""statusCode"": 200, ""b...","{""customerType"": ""个人"", ""customerName"": ""溫先生"", ...",个人,溫先生,,,,...,ER-GD400HK,维修,维修,燈膽燒了,維修法,,,,,4.197126
1,289,9bdd9ccf-d94b-4f38-8a5d-35e4dd43aeee,"agent: 咁嘅\ncustomer: 你好呀,我正在用你們的抽濕機\ncustomer:...","_______costTime:13034ms;{""statusCode"": 200, ""b...","{""customerType"": ""个人"", ""customerName"": ""余先生"", ...",个人,余先生,90997910,90256922,,...,RADY200H,维修,维修,濕濕咗嘅時候呢嗰啲水呢直接流落去地下接不到那些水,,,,,,4.937352
2,290,8fcf8eb4-d4da-4146-a42c-d3ba711b4ed0,customer: 夭壽\nagent: 哦\ncustomer: 雞腿\ncustomer...,"_______costTime:16609ms;{""statusCode"": 200, ""b...","{""customerType"": """", ""customerName"": ""梁先生"", ""p...",,梁先生,,,,...,C-A-L-O-I-E-8-H,,,,,,,,,4.351078
3,291,aec95ea8-e448-45a6-be21-63a6b6326b9f,agent: 嗨\ncustomer: 你好呀\ncustomer: 今日呢就買咗架洗衣機嘅...,"_______costTime:12588ms;{""statusCode"": 200, ""b...","{""customerType"": """", ""customerName"": """", ""phon...",,,6252526,,,...,,维修,维修,漏水,维修,,,,,3.245518
4,292,942df28b-6803-4795-ab8b-7cba16a92ea8,"customer: 喂,駱小姐你好。\nagent: 早晨鐘之後會唔會報我姓駱,請問有咩幫到...","_______costTime:19305ms;{""errorMessage"": ""subs...","{""customerType"": """", ""customerName"": ""陳小姐"", ""p...",,陳小姐,9684-9108,,,...,BL85AEH(WW),维修,,洗衣机洗完衣服后有黑色一片片的东西,,,,,,5.128786


In [48]:
all_columns = df.columns.tolist()

# First 4 columns to remain in the front
fixed_columns = ['param_id', 'contact_id', 'content', 'extracted_content', 'extracted_info_qwen2', 'extracted_info_sonnet']

# Remaining columns
remaining_columns = [col for col in all_columns if col not in fixed_columns]

# Group remaining columns by their model prefix
grouped_columns = {}
for col in remaining_columns:
    prefix = col.split('_')[0]  # assuming model prefix is always second part after split by '_'
    if prefix not in grouped_columns:
        grouped_columns[prefix] = []
    grouped_columns[prefix].append(col)

# Create the new column order
new_column_order = fixed_columns
for prefix in grouped_columns: # sorted(grouped_columns):
    new_column_order.extend(grouped_columns[prefix])

# Reassign columns to the DataFrame
df = df[new_column_order]
df.head()

,param_id,contact_id,content,extracted_content,extracted_info_qwen2,extracted_info_sonnet,address_qwen2,address_sonnet,customerName_qwen2,customerName_sonnet,...,serialNumber_qwen2,serialNumber_sonnet,srSubType_qwen2,srSubType_sonnet,srType_qwen2,srType_sonnet,symptomDescription_qwen2,symptomDescription_sonnet,time_cost_qwen2,time_cost_sonnet
0,288,3ee4b9f4-7674-4978-9066-d89b46c9adb4,"agent: 主持人,多謝您的訪問,我們下集見,再見!\ncustomer: 喂,嗨,嗨,我...","_______costTime:21508ms;{""statusCode"": 200, ""b...","{""customerType"": ""个人"", ""customerName"": ""溫先生"", ...","{\n ""customerType"": ""个人"",\n ""customerName"": ...","楊逸居第三座,九樓,A7","楊逸居第三座,九樓,A7",溫先生,溫先生,...,ER-GD400HK,ER-GD400HK,维修,维修,维修,维修,燈膽燒了,燈膽燒了,9.715980,4.197126
1,289,9bdd9ccf-d94b-4f38-8a5d-35e4dd43aeee,"agent: 咁嘅\ncustomer: 你好呀,我正在用你們的抽濕機\ncustomer:...","_______costTime:13034ms;{""statusCode"": 200, ""b...","{""customerType"": ""个人"", ""customerName"": ""余先生"", ...","{""customerType"": ""个人"", ""customerName"": ""余生"", ""...",柴灣道111號高威港第四座四座八樓私宅,柴灣道111號高威港四座八樓私宅,余先生,余生,...,RADY200H,RADY200H,,维修,维修,维修,"濕濕咗嘅時候呢嗰啲水呢直接流落去地下, 接不到那些水",濕濕咗嘅時候呢嗰啲水呢直接流落去地下接不到那些水,8.363742,4.937352
2,290,8fcf8eb4-d4da-4146-a42c-d3ba711b4ed0,customer: 夭壽\nagent: 哦\ncustomer: 雞腿\ncustomer...,"_______costTime:16609ms;{""statusCode"": 200, ""b...","{""customerType"": """", ""customerName"": ""梁先生"", ""p...","{""customerType"": ""个人"", ""customerName"": """", ""ph...",油塘高麟阁410室,高倫閣,梁先生,,...,"GR字头, R28",C-A-L-O-I-E-8-H,,,,,,,8.174167,4.351078
3,291,aec95ea8-e448-45a6-be21-63a6b6326b9f,agent: 嗨\ncustomer: 你好呀\ncustomer: 今日呢就買咗架洗衣機嘅...,"_______costTime:12588ms;{""statusCode"": 200, ""b...","{""customerType"": """", ""customerName"": """", ""phon...","{""customerType"": ""个人"", ""customerName"": ""溫先生"", ...","天龍村,公安路",,,溫先生,...,,,,维修,维修,维修,漏水,漏水,5.971399,3.245518
4,292,942df28b-6803-4795-ab8b-7cba16a92ea8,"customer: 喂,駱小姐你好。\nagent: 早晨鐘之後會唔會報我姓駱,請問有咩幫到...","_______costTime:19305ms;{""errorMessage"": ""subs...","{""customerType"": """", ""customerName"": ""陳小姐"", ""p...","{""customerType"": ""个人"", ""customerName"": """", ""ph...",啟德啟程村欣賞情樓33樓3307,启德启程村欣赏情楼33楼3307,陳小姐,,...,BW-BL85A2,BL85AEH(WW),,,,维修,洗衣機洗完衣服後黏上黑色一塊塊的東西,洗衣机洗完衣服后有黑色一片片的东西,10.509946,5.128786


In [49]:
df.to_csv("outputs/对话提取结果.csv", index=False)

## Llama3-72B 